In [4]:
!git clone https://github.com/xlsrln/cat

fatal: destination path 'cat' already exists and is not an empty directory.


In [10]:
import requests
from io import StringIO
import pandas as pd
import datetime
from time import sleep

def read_df(string, sep="\t"):
    return pd.read_csv(StringIO(string), sep=sep)

car_lookup = pd.read_csv("cat/resources/car_lookup.csv")
stage_lookup = pd.read_csv("cat/resources/stage_lookup.csv")

champ_class = "GroupB"

root_url = "https://www.funselektorfun.com/artofrally/leaderboard/"

with open('cat/resources/cat_stages.txt') as f:
    champ_rally = eval(f.read())
    f.close()

In [11]:
def format_time(x):
    sign = ""
    
    if x < 0:
        sign = "-"
        x = -x
    elif x == 0:
        return("")

    ob = datetime.datetime.fromtimestamp(x/1000.0)
    return sign + f"00:{ob.minute:02}:{ob.second:02}.{round(ob.microsecond/1000):03}"

def get_wr_and_top10(stage):
    request_str = root_url + stage + "/0/0"
    response = requests.get(request_str)

    temp = top10_time = eval(response.text)['leaderboard']

    wr_time = temp[0]['score']
    top10_time = temp[-1]['score']

    return wr_time, top10_time

def get_rally_df(event, user=None):
    times = []
    for n in range(len(champ_rally[event]['stages'])):
        x = champ_rally[event]['stages'][n]
        w = champ_rally[event]['weather'][n]
        c = champ_rally[event]['country']

        ws = 'Wet' if w in ['rain', 'snow'] else 'Dry'

        stage = stage_lookup.set_index('stage_name').to_dict()['stage'][x]
        stage_str = '_'.join([c, stage, ws, champ_class])

        # get wr and top 10 times
        wr_time, top10_time = get_wr_and_top10(stage_str)

        vals = [c, n, x, w, stage_str, wr_time, top10_time]
        cols = ['country', 'no', 'stage_name', 'weather', 'stage_str', 
                  'wr', 'top10', ]

        # collect
        times.append((x for x in vals))

    df = pd.DataFrame(times)
    df.columns = cols

    df['wr_time'] = df.wr.apply(format_time)
    df['top10_time'] = df.top10.apply(format_time)

    return df


def rally_stats(country):
    df = get_rally_df(country)
    
    print("\nstages:")
    display(df[['stage_name', 'weather', 'wr_time', 'top10_time', ]])

    print("\nworld rec time: " + format_time(df.wr.sum()))
    print("top10 tot time: " + format_time(df.top10.sum()))

In [12]:
rally_stats('s2_sardinia')


stages:


,stage_name,weather,wr_time,top10_time
0,san benedetto,sunset,00:02:00.745,00:02:03.491
1,montevecchio,night,00:02:16.666,00:02:20.124
2,villacidro,night,00:01:47.550,00:01:50.267
3,gennamari,night,00:02:21.964,00:02:27.451
4,san gavino monreale,night,00:01:19.471,00:01:21.966



world rec time: 00:09:46.396
top10 tot time: 00:10:03.299
